In [1]:
from aiida import load_profile, orm
## Load your profile here
load_profile('bandgap') 
from aiida.plugins import WorkflowFactory
from aiida.engine import submit

In [2]:
qb = orm.QueryBuilder()
qb.append(orm.StructureData, filters={'id':{'in':[4182]}})
qb.append(WorkflowFactory('quantumespresso.ocv.ocvwc'), tag='smearing_pw')
qb.append(WorkflowFactory('quantumespresso.pw.relax'))
qb.append(orm.StructureData)
qb.all(flat=True)

[<StructureData: uuid: b9af201f-1b55-4446-8e19-b66f0c8e1afc (pk: 96967)>,
 <StructureData: uuid: e0356f5c-114b-4251-9ea2-e8b78f0440f1 (pk: 97168)>,
 <StructureData: uuid: cc4cdb94-8ded-415c-9706-c918a1408298 (pk: 97242)>]

In [5]:
submit_OCV_workchain(orm.load_node(4182), code, overrides, qb.all(flat=True)[-2], qb.all(flat=True)[-1])

<WorkChainNode: uuid: 7ac6e87a-cdf0-4c10-bac5-654f9f7ddc66 (pk: 97418) (aiida.workflows:quantumespresso.ocv.ocvwc)>

In [3]:
code = orm.load_code(label='pw-qe-6.5-sirius')
overrides = {'pseudo_family': 'SSSP/1.1.2/PBEsol/efficiency'}
## Use the bundled test structure here
structure = orm.load_node('4522933d-83d8-4aba-a3a2-425a48cce792')

In [4]:
## Submitting the OCVWorkChain
def submit_OCV_workchain(structure, code, overrides, discharged_unitcell_relaxed, charged_unitcell_relaxed, time=3600, num_machines=2, num_mpiprocs_per_machine=12, num_cores_per_mpiproc=1):

    OCVWorkChain = WorkflowFactory('quantumespresso.ocv.ocvwc')

    builder = OCVWorkChain.get_builder_from_protocol(code, structure=structure, overrides=overrides, discharged_unitcell_relaxed=discharged_unitcell_relaxed, charged_unitcell_relaxed=charged_unitcell_relaxed)    

    builder.update({'clean_workdir':orm.Bool(False)})

    builder.ocv_relax['base']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc

    builder.ocv_relax['base_final_scf']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
    builder.ocv_parameters['distance'] = 4
    
    ## LFPO specific parameters
    # builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['smearing'] = 'gauss'
    # builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['degauss'] = 0.005 
    # builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['starting_magnetization(1)'] = 0.5
    # builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['starting_magnetization(2)'] = -0.5

    # builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['diagonalization'] = 'david'
    # builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['mixing_mode'] = 'local-TF'
    # builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['mixing_beta'] = 0.1
    # builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 500

    # builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['smearing'] = 'mv'
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['degauss'] = 0.005 
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['starting_magnetization(1)'] = 0.5
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['starting_magnetization(2)'] = -0.5

    # builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['diagonalization'] = 'david'
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['mixing_mode'] = 'local-TF'
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['mixing_beta'] = 0.1
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 700
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['conv_thr'] = 5e-8

    return submit(builder)